In [118]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import make_moons

In [123]:

## Generating data
x, y = make_moons(n_samples=1000, noise=0.1)

In [124]:
x_plus_bias=np.c_[np.ones((m,1)),x]

In [115]:
from sklearn.preprocessing import StandardScaler
m,n=x.shape
std_scaler=StandardScaler()
scaled_x=std_scaler.fit_transform(x)
scaled_x_plus_bias=np.c_[np.ones((m,1)),scaled_x]

In [125]:

## Construct a computational graph for logistic regression

from datetime import datetime
import numpy.random as rnd
tf.reset_default_graph()
epsilon = 1e-7
n_epochs=1000
batch_size=50
learning_rate=1e-2
n_batches=int(np.ceil(m/batch_size))

now=datetime.now().strftime('%Y%m%d%H%M%S')
root_logdir='moons'
logdir='{}/run/-{}'.format(root_logdir,now)


def fetch_batch(epoch,batch_index,batch_size):
    rnd.seed(epoch)
    indices=rnd.permutation(m)
    if batch_index==n_batches-1:
        x_batch=x_plus_bias[batch_index*batch_size:]
        y_batch=y.reshape(-1,1)[batch_index*batch_size:]
    else:
        x_batch=x_plus_bias[batch_index*batch_size:batch_index*batch_size+batch_size]
        y_batch=y.reshape(-1,1)[batch_index*batch_size:batch_index*batch_size+batch_size]
    return x_batch, y_batch


def logistic_regression():
    X=tf.placeholder(tf.float32,shape=[None,n+1],name='X')
    Y=tf.placeholder(tf.float32,shape=[None,1],name='Y')
    w=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),dtype=tf.float32,name='weights')
    logits=1./(1.+tf.exp(-tf.matmul(X,w)))
    cross_entropy=tf.losses.log_loss(Y, logits)
    return X,Y,w,logits,cross_entropy
    
X,Y,w,logits,cross_entropy=logistic_regression()
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(cross_entropy)


cross_entropy_summary=tf.summary.scalar('entropy',cross_entropy)
File_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())

init=tf.global_variables_initializer()
saver=tf.train.Saver()

In [126]:
## Perform training

with tf.Session() as sess:
    sess.run(init)
    
    
    for epoch in range(n_epochs):
        if epoch%100==0:
            save_path=saver.save(sess,"./tmp/my_model_final.ckpt")
            print('entropy:', cross_entropy.eval(feed_dict={X:x_batch,Y:y_batch}))
        for batch_index in range(n_batches):
            x_batch,y_batch=fetch_batch(epoch,batch_index,batch_size)
            if batch_index%10==0:
                summary_str=cross_entropy_summary.eval(feed_dict={X:x_batch,Y:y_batch})
                step=batch_index+1+epoch*n_batches
                File_writer.add_summary(summary_str,step)
            sess.run(training_op,feed_dict={X:x_batch,Y:y_batch})
    weights=w.eval()
    print(weights)
    save_path=saver.save(sess,"./tmp/my_model_final.ckpt")

entropy: 0.986466
entropy: 0.346655
entropy: 0.314789
entropy: 0.30527
entropy: 0.302294
entropy: 0.301828
entropy: 0.302402
entropy: 0.303407
entropy: 0.30457
entropy: 0.30576
[[ 0.65127939]
 [ 1.14652693]
 [-4.98445845]]
